https://esdac.jrc.ec.europa.eu/content/lucas2015-topsoil-data

In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} openpyxl folium matplotlib mapclassify

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# LUCAS TOPSOIL 2009 datasheets
lts_09 = r"D:\soil_data\LUCAS_TOPSOIL_v1\LUCAS_TOPSOIL_v1.xlsx"
lts_15 = r"D:\soil_data\LUCAS2015_topsoildata_20200323\LUCAS_Topsoil_2015_20200323.csv"
lts_DE = r'D:\soil_data\LUCAS2015_topsoildata_20200323_micro\LUCAS_Topsoil_2015_DE.csv'

In [3]:
# LUCAS TOPSOIL 2009
# Load excel file with sample_ID as index.
df_09 = pd.read_excel(lts_09)
# Unselect rows with invalid POINT_ID.
df_09 = df_09[df_09['POINT_ID'].astype(str).str.isdigit()]
# Set POINT_ID as index.
df_09.set_index("POINT_ID", inplace=True)
# Rename to consider equality between 2009 and 2015 data
df_09.rename(index={'POINT_ID': 'Point_ID'}, inplace=True) 
# Add columns from 2015 survey 
df_09[['Elevation','LC1', 'LU1', 'Soil_Stones', "NUTS_0","NUTS_1","NUTS_2","NUTS_3", "LC1_Desc","LU1_Desc"]] = np.nan

In [4]:
# LUCAS TOPSOIL 2015
df_15 = pd.read_csv(lts_15, index_col="Point_ID")
# Update missing landcover information from the 2015 survey
df_09.update(df_15)
#Reset Index 
df_09.reset_index(inplace=True)

In [5]:
# German Points Ides
df_DE = pd.read_csv(lts_DE, index_col="Point_ID")
DE_Points = list(df_DE.index)

In [6]:
# Creating a GeoDataFrame from a DataFrame with coordinates
df_09 = gpd.GeoDataFrame(df_09, geometry=gpd.points_from_xy(df_09.GPS_LONG, df_09.GPS_LAT), crs=4326)
# Load area of interest into geopandas for masking df_09
aoi = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\oberrheingraben_shp\oberrheingraben-polygon.shp').to_crs(epsg=4326)
# Subset data
df_grassland = df_09[df_09.LC1_Desc == 'Grassland without tree/shrub cover']
df_grassland_DE = df_grassland[df_grassland['POINT_ID'].isin(DE_Points)]
df_bare = df_09[df_09.LC1_Desc == 'OTHER BARE SOIL']
df_bare_DE = df_bare[df_bare['POINT_ID'].isin(DE_Points)]
df_DE = df_09[df_09['POINT_ID'].isin(DE_Points)]
# Save data to disk 
df_grassland.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_grassland.csv', index=False)
df_grassland_DE.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_grassland_DE.csv', index=False)
df_bare.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_bare.csv', index=False)
df_bare_DE.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_bare_DE.csv', index=False)
df_DE.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/LTS_DE.csv', index=False)


In [ ]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
greece = world.loc[world['name'] == 'Germany']

In [7]:
from irrigation_detection import get_s1_s2_era5_df
from glob import glob

In [ ]:
ried_soil_samples = pd.DataFrame({'POINT_ID' : [1,2,3,4,5,6,7], 
                                  'longitude' : [8.5332, 8.5225, 8.4396, 8.4187, 8.4499, 8.4191, 8.84081],
                                 'latitude' : [49.9083, 49.8843, 49.9201, 49.8961, 49.8847, 49.8722, 49.8471],
                                 'silt' : [13.0, 21, 56.7, 66.8, 63.5, 67.1, 74.2],
                                 'sand' : [85.1, 74.8, 40.2, 31.5, 35.5, 32.1, 22.1],
                                 'coarse' : [1.9, 4.2, 2.9, 1.7, 1.0, 0.7, 3.7]})
ried_soil_samples['clay'] = 100 - ried_soil_samples.silt - ried_soil_samples.sand
ried_soil_samples.to_csv(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/ried_soil_samples.csv')


In [3]:
ried_full_points = gpd.read_file('C:/Users/USER/Desktop/Master_Irrigation/03_GIS/landcover/ried_points_lc_30_40.geojson')

In [ ]:
longitude = list(ried_soil_samples.longitude)
latitude = list(ried_soil_samples.latitude)
point_id = list(ried_soil_samples.POINT_ID)

In [10]:
longitude = list(df_DE.geometry.x)
latitude = list(df_DE.geometry.y)
point_id = list(df_DE.POINT_ID)

In [8]:
#filepath_grassland = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland/'
#filepath_grassland_DE = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/grassland_DE/'
#filepath_bare_soil = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/bare_soil/'
filepath_DE = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/DE/'
#filepath_ried = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/ried_soil_samples/'
#filepath_ried_10m = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/ried_soil_samples_10m/'
#filepath_ried_fullpoints_30m = r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/ried_fullpoints_30m_17-19/'

files = glob(filepath_DE + '*')
existing_ids = [int(x.split('_')[-3]) for x in files]

In [ ]:
for lon, lat, idx in zip(longitude, latitude, point_id):
    if idx not in existing_ids:
        get_s1_s2_era5_df(
            longitudes = [lon],
            latitudes = [lat],
            polygon = None,
            ids = [idx], 
            filepath = filepath_DE,
            start = '2017-01-01', 
            end = '2020-01-01',
            scale_s1 = 30,
            scale_s2 = 30,
            scale_era5 = 30,
            idx_name = 'POINT_ID',
            get_grid_scale = False
        )
    else:
        print(f'{idx} data already exists... skip download!')

46643144 data already exists... skip download!
46623118 data already exists... skip download!
46603108 data already exists... skip download!
46563124 data already exists... skip download!
46543152 data already exists... skip download!
46543138 data already exists... skip download!
46483136 data already exists... skip download!
46443178 data already exists... skip download!
46443124 data already exists... skip download!
46423144 data already exists... skip download!
46423126 data already exists... skip download!
46403220 data already exists... skip download!
46403170 data already exists... skip download!
46403134 data already exists... skip download!
46403130 data already exists... skip download!
46383154 data already exists... skip download!
46363218 data already exists... skip download!
46363192 data already exists... skip download!
46323180 data already exists... skip download!
46323144 data already exists... skip download!
46323140 data already exists... skip download!
46323122 data

S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/DE/POINT_ID_45063310_12.74773_52.86725.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 12.72758, latitude: 52.54396, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/DE/POINT_ID_45063274_12.72758_52.54396.geojson  succesfully to disk
Finish downloads... failed to download []
Download Sentinel 1 & Sentinel 2 & ERA5 data for longitude: 12.66713, latitude: 51.53821, Item: 0/0
S1 data collection succseed!
S2 data collection sucseed!
ERA5 data collection sucseed!
Write : C:/Users/USER/Desktop/Master_Irrigation/03_GIS/soil_classification/download_ts/DE/POINT_ID_45063162_12.66713_51.53821.geojson  succesfull

https://keras.io/examples/timeseries/timeseries_classification_from_scratch/

In [ ]:
array(['Shrubland without tree cover', nan,
       'Shrubland with sparse tree cover', 'Olive groves',
       'Pine dominated coniferous woodland',
       'Pine dominated mixed woodland',
       'Spontaneously re-vegetated surfaces', 'Sunflower',
       'Other root crops', 'Common wheat', 'Broadleaved woodland',
       'Cotton', 'Other mixed woodland', 'Durum wheat', 'Oranges',
       'Clovers', 'Lucerne', 'Maize', 'Inland fresh running water',
       'Tobacco', 'Other fruit trees and berries', 'Soya',
       'Other coniferous woodland', 'Nuts trees', 'Vineyards',
       'OTHER BARE SOIL', 'Other non-permanent industrial crops',
       'Tomatoes', 'Grassland without tree/shrub cover', 'Oats',
       'Mix of cereals', 'Grassland with sparse tree/shrub cover', 'Rye',
       'Barley', 'Rape and turnip rape', 'Dry pulses',
       'Spruce dominated coniferous woodland',
       'Spruce dominated mixed woodland',
       'Other Leguminous  and mixtures for fodder', 'Cherry fruit',
       'Other fresh vegetables', 'Temporary grassland', 'Triticale',
       'Apple fruit', 'Potatoes', 'Nurseries', 'Sugar beet',
       'Non built-up linear features', 'OTHER ARTIFICIAL AREAS',
       'Other cereals', 'LICHENS AND MOSS', 'SAND', 'Peatbogs',
       'Strawberries', 'Inland marshes', 'Pear fruit',
       'Other fibre and oleaginous crops', 'Other citrus fruit',
       'Permanent industrial crops', 'Rice',
       'Floriculture and ornamental plants', 'ROCKS AND STONES',
       'Inland salty water bodies', 'Inland fresh water bodies',
       'Salines', 'Non built-up area features'], dtype=object)

In [ ]:
#Subset points from Hessisches Ried
p_id = [42142960, 42122956, 42142970, 42082950]
df_p = df_09[df_09.POINT_ID.isin(p_id)]

In [ ]:
#Export Data
#df_09_subset.to_csv(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_subset.csv')
#df_09_subset.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.shp')
#df_09_subset.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.kml')
#df_p.to_csv(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_p_ried.csv')